In [1]:
---

## ۲. نسخه پایتون با شرط خطا برای جلوگیری از اجرا

`python
raise Exception("⚠️ این کد فقط برای نمایش است و اجرا نمی‌شود!")

import os
import pdfplumber
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy

print("بارگذاری مدل SciSpacy ...")
nlp_spacy = spacy.load("en_ner_bionlp13cg_md")

print("بارگذاری مدل d4data/biomedical-ner-all ...")
model_id = "d4data/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(model_id)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

pdf_folder = "D:/P1"

results = []

def split_text(text, max_len=512):
    return [text[i:i+max_len] for i in range(0, len(text), max_len)]

print("شروع پردازش فایل‌های PDF ...")

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        print(f"\n📄 در حال پردازش: {filename}")
        filepath = os.path.join(pdf_folder, filename)
        with pdfplumber.open(filepath) as pdf:
            full_text = ''
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text

        keywords = ["results", "funding", "findings", "interpretation", "conclusion"]
        selected_text = ''
        for kw in keywords:
            idx = full_text.lower().find(kw)
            if idx != -1:
                selected_text += full_text[idx:idx+2000]
        if not selected_text:
            selected_text = full_text[:2000]

        doc = nlp_spacy(selected_text)
        spacy_entities = set([ent.text for ent in doc.ents])
        print(f"🔬 SciSpacy: {len(spacy_entities)} بیومارکر پیدا شد.")

        ents = []
        for chunk in split_text(selected_text):
            ents.extend(ner_pipeline(chunk))
        d4data_entities = set([ent['word'] for ent in ents])
        print(f"🤖 d4data NER: {len(d4data_entities)} بیومارکر پیدا شد.")

        results.append({
            "Filename": filename,
            "SciSpacy Biomarkers": ", ".join(spacy_entities),
            "BiomedNLP Biomarkers": ", ".join(d4data_entities)
        })

print("\n📦 در حال ذخیره نتایج در فایل‌ها ...")
df = pd.DataFrame(results)
df.to_excel("Extracted_Biomarkers.xlsx", index=False)
df.to_csv("Extracted_Biomarkers.csv", index=False)

print("✅ استخراج و ذخیره اطلاعات تمام شد.\n🗂 فایل‌ها: Extracted_Biomarkers.xlsx و Extracted_Biomarkers.csv")

SyntaxError: invalid syntax (37484045.py, line 1)